In [1]:
import os
import re

import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [140]:
train_data = pd.read_csv( 'data_clean.csv' )
train_data = train_data[train_data['text'].notnull()]

In [141]:
train_data

,text,sentiment
0,according gran company plans move production r...,neutral
1,technopolis plans develop stages area less squ...,neutral
2,international electronic industry company elco...,negative
3,new production plant company would increase ca...,positive
4,according company updated strategy years baswa...,positive
...,...,...
4841,london marketwatch share prices ended lower lo...,negative
4842,rinkuskiai beer sales fell per cent million li...,neutral
4843,operating profit fell eur mn eur mn including ...,negative
4844,net sales paper segment decreased eur mn secon...,negative


In [142]:
text = list(train_data['text'])
sentiments = list(train_data['sentiment'])

In [143]:
for i in range(len(sentiments)):
    if sentiments[i]=='neutral':
        sentiments[i]=1
    elif sentiments[i]=='negative':
        sentiments[i]=0
    elif sentiments[i]=='positive':
        sentiments[i]=2

In [144]:
sentences = []
for t in text :
    sentences.append(t.split())

In [180]:
num_features = 36
min_word_count = 3
num_workers = 3    
context = 10          
downsampling = 0.001 

In [181]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',
   level=logging.INFO)

In [182]:
from gensim.models import word2vec

model = word2vec.Word2Vec(sentences, workers=num_workers,
           size=num_features, min_count = min_word_count,
            window = context, sample = downsampling)

2020-06-14 01:33:24,594 : INFO : collecting all words and their counts
2020-06-14 01:33:24,595 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-06-14 01:33:24,606 : INFO : collected 8951 word types from a corpus of 59521 raw words and 4845 sentences
2020-06-14 01:33:24,607 : INFO : Loading a fresh vocabulary
2020-06-14 01:33:24,614 : INFO : effective_min_count=3 retains 3134 unique words (35% of original 8951, drops 5817)
2020-06-14 01:33:24,615 : INFO : effective_min_count=3 leaves 52362 word corpus (87% of original 59521, drops 7159)
2020-06-14 01:33:24,623 : INFO : deleting the raw counts dictionary of 8951 items
2020-06-14 01:33:24,624 : INFO : sample=0.001 downsamples 34 most-common words
2020-06-14 01:33:24,625 : INFO : downsampling leaves estimated 47120 word corpus (90.0% of prior 52362)
2020-06-14 01:33:24,640 : INFO : estimated required memory for 3134 words and 36 dimensions: 2469592 bytes
2020-06-14 01:33:24,641 : INFO : resetting layer weight

In [183]:
def get_features(words, model, num_features):
    feature_vector = np.zeros((num_features),dtype=np.float32)

    num_words = 0
    index2word_set = set(model.wv.index2word)

    for w in words:
        if w in index2word_set:
            num_words += 1
            feature_vector = np.add(feature_vector, model[w])

    feature_vector = np.divide(feature_vector, num_words)
    return feature_vector

In [184]:
def get_dataset(reviews, model, num_features):
    dataset = list()

    for s in reviews:
        dataset.append(get_features(s, model, num_features))

    reviewFeatureVecs = np.stack(dataset)
    
    return reviewFeatureVecs

In [185]:
test_data_vecs = get_dataset(sentences, model, num_features)

c:\users\wjdwn\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.
c:\users\wjdwn\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in true_divide
  if sys.path[0] == '':


In [186]:
arr=[]
for i in range(len(test_data_vecs)):
    for j in range(len(test_data_vecs[i])):
        if np.isnan(test_data_vecs[i][j]):
            print(i,j)
            if i not in arr:
                arr.append(i)

3449 0
3449 1
3449 2
3449 3
3449 4
3449 5
3449 6
3449 7
3449 8
3449 9
3449 10
3449 11
3449 12
3449 13
3449 14
3449 15
3449 16
3449 17
3449 18
3449 19
3449 20
3449 21
3449 22
3449 23
3449 24
3449 25
3449 26
3449 27
3449 28
3449 29
3449 30
3449 31
3449 32
3449 33
3449 34
3449 35
4804 0
4804 1
4804 2
4804 3
4804 4
4804 5
4804 6
4804 7
4804 8
4804 9
4804 10
4804 11
4804 12
4804 13
4804 14
4804 15
4804 16
4804 17
4804 18
4804 19
4804 20
4804 21
4804 22
4804 23
4804 24
4804 25
4804 26
4804 27
4804 28
4804 29
4804 30
4804 31
4804 32
4804 33
4804 34
4804 35


In [187]:
arr

[3449, 4804]

In [188]:
X=[]
y=[]
for i in range(len(test_data_vecs)):
    if i not in arr:
        X.append(test_data_vecs[i])
        y.append(sentiments[i])

In [189]:
from sklearn.model_selection import train_test_split
import numpy as np

RANDOM_SEED = 42
TEST_SPLIT = 0.2

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SPLIT, random_state=RANDOM_SEED)

In [190]:
from sklearn.linear_model import LogisticRegression

lgs = LogisticRegression(class_weight='balanced')
lgs.fit(X_train, y_train)

LogisticRegression(class_weight='balanced')

In [191]:
predicted = lgs.predict(X_test)
print("Accuracy: %f" % lgs.score(X_test, y_test))

Accuracy: 0.565531
